In [7]:
!pip3 install matplotlib --user

In [10]:
import tensorflow as tf
tf.enable_eager_execution()

import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL

import time

from IPython import display
import warnings
warnings.filterwarnings('ignore')

In [11]:
(train_images,train_labels),(_,_)=tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 5s 0us/step


In [15]:
train_images=train_images.reshape(train_images.shape[0],28,28,1).astype('float32')
train_images=(train_images-127.5)/127.5 #Normalize

In [16]:
BUFFER_SIZE=60000
BATCH_SIZE=256

In [17]:
train_dataset=tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [19]:
def make_generator_model():
    model=tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(7*7*256,use_bias=False,input_shape=(100,)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Reshape((7,7,256)))
    assert model.output_shape==(None,7,7,256)#None is batch size
    
    model.add(tf.keras.layers.Conv2DTranspose(128,(5,5),strides=(1,1),padding='same',use_bias=False))
    assert model.output_shape==(None,7,7,128)
    model.add(tf.keras.layers.BatchNormalization())
    mode.add(tf.keras.layers.LeakyReLU())
    
    model.add(tf.keras.layers.Conv2DTranspose(64,(5,5),strides=(2,2),padding='same',use_bias=False))
    assert model.output_shape==(None,14,14,64)
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    
    model.add(tf.keras.layers.Conv2DTranspose(1,(5,5),strides=(2,2),padding='same',use_bias=False))
    assert model.output_shape==(28,28,1)
    